# wikipedia Optional
いちいちグラフの読み込みに時間がかかるのでこれだけipynbにしました。

## グラフの読み込み

In [2]:
import copy
import random
def read_graph_data(linksfile,namesfile):
    lf = open(linksfile)
    nf = open(namesfile)
    names=dict([line.strip().split() for line in nf.readlines()])
    nodelist={v:[] for v in names.values()} #dict
    for line in lf.readlines():
        n_from,n_to=line.strip().split()
        
        nodelist[ names[n_from] ].append(names[n_to])
    lf.close()
    nf.close()
    return nodelist



In [3]:
#グラフの読み込み
graph=read_graph_data("./wikipedia_links/links.txt","./wikipedia_links/pages.txt")
print("読み込み完了")


読み込み完了


## 任意の2頂点間の最短経路を調べる

In [22]:
#adrianからあなたに行けるか？
def isConnected_BFS(graph,A,B): #AとBがグラフ上で連結しているかと、最短距離をBFSでもとめる
    #BFSは重みなしグラフの単一始点最短経路問題において最も効率が良いアルゴリズムである。
    if A not in graph or B not in graph:
        return "そのような人はいません"
    now=[A,] 
    visited={ k:0 for k in graph.keys()}
    visited[A]=1
    count=0
    if A==B:
        return count
    while len(now)>0:
        count+=1
        nxt=[]
        for i in now:
            for next_node in graph[i]: 
                if next_node==B:
                    return str(count)+"回辺を辿れば到達できます"
                if  visited[next_node]==0:
                    visited[next_node]=1
                    nxt.append(next_node)
        now=nxt
    return "到達できません"

In [23]:
isConnected_BFS(graph,"渋谷","")

'2回辺を辿れば到達できます'

## 小さいグラフの作成

In [4]:
def collect_around(graph,start,N):
    if start not in graph:
        print("ERROR:そのような記事はありません")
        exit(1)
    now=[start,] 
    around=[start,]
    visited={ k:0 for k in graph.keys()}
    visited[start]=1
    for i in range(N):
        nxt=[]
        for i in now:
            for next_node in graph[i]: 
                if  visited[next_node]==0:
                    visited[next_node]=1
                    around.append(next_node)
                    nxt.append(next_node)
        now=nxt
    return around

def make_small_graph(graph,selected):
    newgraph={k:[] for k in selected}
    for frm in selected:
        for to in graph[frm]:
            if to in selected:
                newgraph[frm].append(to)
    return newgraph



In [7]:
#小さなグラフをつくる（計算量が大きいので）
around=collect_around(graph,"渋谷",1)
print(len(around))
around_graph=make_small_graph(graph,around)
print(len(around_graph['音楽']))


322
5


## 媒介中心性の計算、コミュニティ検出

In [9]:
import queue
def count_betweeness_Brandes(graph):#ノード、辺の媒介中心性の計算
    edge_betweeness={}#(from,to):edge betweeness
    for frm,tos in graph.items():#全てのノードを追加
        for to in tos:
            edge_betweeness[(frm,to)]=0
    #algorithm:https://www.eecs.wsu.edu/~assefaw/CptS580-06/papers/brandes01centrality.pdf
    #は頂点の媒介中心性を求めているので、それを辺にも応用したもの
    #ちなみにノードの中心媒介性はそのまま用いてもノードの重要度の指標になりそう。
    C_b={k:0 for k in graph.keys()}
    for s in graph.keys():
        S=[] #stack
        P={k:[] for k in graph.keys()} #P[v]:頂点vに向かう最短経路において前に通った頂点のリスト
        sigma={k:0 for k in graph.keys()}#最短経路の数
        sigma[s]=1
        d={k:-1 for k in graph.keys()}#P[v]:sからvへの最短距離
        d[s]=0
        #ここは普通にBFSを行っている
        Q=queue.Queue()
        Q.put(s)
        while not Q.empty():
            v=Q.get()
            S.append(v)#sから近い順に（最短距離が単調増加に）格納される
            for w in graph[v]:
                if d[w]<0:
                    Q.put(w)
                    d[w]=d[v]+1
                if d[w]==d[v]+1:
                    sigma[w]+=sigma[v]
                    P[w].append(v)
        delta={k:0 for k in graph.keys()}#これはsを始点とした時の頂点kの媒介中心性が求まる
        delta_edge={k:0 for k in edge_betweeness.keys()}#これはsを始点とした時のedgeの媒介中心性が求まる
        while len(S)>0:
            to=S.pop()#これは必ずsから遠い順に取り出される
            for frm in P[to]:#toの前に通った頂点frmに対して
                delta[frm]+=sigma[frm]/sigma[to]*(1+delta[to])
                #toの媒介中心性(決定済み)にsigma[frm]/sigma[to]をかければ辺の媒介性が求まる
                edge_betweeness[(frm,to)]+=(sigma[frm]/sigma[to])*delta[to]
            if to!=s:
                C_b[to]+=delta[to]
    return C_b,edge_betweeness


def connected_groups(graph_directed):
    #連結であるとは、そのグループの中のどの2頂点を選んでもどちらか一方向に道が存在することとする。
    #無向グラフにしておく（ちょっとアレな手法かもしれない…）
    graph=copy.deepcopy(graph_directed)
    for frm,tos in graph.items():
        for to in tos:
            if frm not in graph[to]:
                graph[to].append(frm)
    groups=[]
    not_connected=[k for k in graph.keys()]
    while len(not_connected)>0:#全てのノードを始点としてBFSを行う
        start=not_connected.pop()
        group=[start,]
        now=[start,] 
        visited={ k:0 for k in graph.keys()}
        visited[start]=1
        count=0
        while len(now)>0:
            count+=1
            nxt=[]
            for i in now:
                for next_node in graph[i]: 
                    if  visited[next_node]==0:
                        group.append(next_node)
                        assert(next_node in not_connected)#無向グラフならこの条件を満たすはず
                        not_connected.remove(next_node)
                        visited[next_node]=1
                        nxt.append(next_node)
            now=nxt
        groups.append(group)
    return groups

#Nグループに分割する（Girvan-Newman法）→授業のグループとの関係が見えるのでは
def grouping_girvan_newman(graph,N):
    new_graph=copy.deepcopy(graph)
    connected_groups_now=connected_groups(new_graph)
    #最初の連結成分
    print("初期状態：")
    print(str(len(connected_groups_now))+"groups",*connected_groups_now,sep='\n')
    while True:
        groups=[]
        #1.残っている全てのリンクのedge betweennessを計算する
        #node_betweenessは、ある頂点が任意の2ノード間の最短パスに含まれる回数。（ただし自分が始点、終点であるものは除く）
        #edge_betweenessは、ある辺が任意の2ノード間の最短パスに含まれる回数
        node_betweeness,edge_betweeness=count_betweeness_Brandes(new_graph)
        #「到達できる場合最短経路長は必ず1である状態」になったらそれ以上の分類は不可能なので
        #辺を切ることをやめる
        if max(node_betweeness.values())==0:
            print("到達できる場合最短経路長は必ず1である状態です。")
            connected_groups_now=connected_groups(new_graph)
            return len(connected_groups_now),connected_groups(new_graph)
        #2.そうでない場合、最もedge betweenessが高いリンクを切る。
        max_edge_from,max_edge_to=max(edge_betweeness, key=edge_betweeness.get)
        #print("removed:",max_edge_from,max_edge_to,edge_betweeness[(max_edge_from,max_edge_to)])
        new_graph[max_edge_from].remove(max_edge_to)
        #3.1-2を、連結成分がN個になるまで繰り返す。
        connected_groups_now=connected_groups(new_graph)
        if len(connected_groups_now)>=N:
            return len(connected_groups_now),connected_groups_now


In [10]:
print("最も媒介中心性が高い記事を選びました。")
node_betweeness,edge_betweeness=count_betweeness_Brandes(around_graph)
important_article=max(node_betweeness, key=node_betweeness.get)
print(important_article)

最も中心媒介性が高い記事を選びました。
渋谷


(渋谷が最も高いのそれはそう、なぜなら渋谷の周りの記事を集めたのだからおそらくほぼ全ての最短路において渋谷を経由しているから)


In [14]:
print("媒介中心性が高い記事をいくつか選びました。")
print(*sorted(node_betweeness.items(), key=lambda x:-x[1])[0:10],sep='\n')

媒介中心性が高い記事をいくつか選びました。
('渋谷', 41584.21016569537)
('渋谷区', 10597.363803706068)
('東京都', 9352.71009692459)
('日本', 7026.995315438036)
('渋谷駅', 5892.528568959307)
('東京都区部', 5514.276347483476)
('東京', 4261.079462614647)
('江戸時代', 2222.977703596916)
('新宿', 1741.6214470517154)
('山手線', 1471.6020554648596)


我々に馴染みの深い記事が選ばれており、かなりいい結果に見える。



------
…ここより以下のコードは計算量が大きすぎて実行がなかなか終わらないのでまだ試していません

もう少し大きなグラフで試してみよう。

In [17]:
#小さなグラフをつくる（計算量が大きいので）
around_bigger=collect_around(graph,"渋谷",2)
print(len(around_bigger))
around_graph_bigger=make_small_graph(graph,around_bigger)
print(len(around_graph_bigger['音楽']))
node_betweeness,edge_betweeness=count_betweeness_Brandes(around_graph_bigger)
important_article=max(node_betweeness, key=node_betweeness.get)
print("媒介中心性が高い記事をいくつか選びました。")
print(*sorted(node_betweeness.items(), key=lambda x:-x[1])[0:10],sep='\n')

39374


KeyboardInterrupt: 

 ## クラスタリング


In [15]:
print("Divide members to N groups:")
N=int(input("N:"))
groupnum,groups=grouping_girvan_newman(graph,N)
print(groupnum,"groups:")
print(*groups,sep='\n')

Divide members to N groups:
N:10


KeyboardInterrupt: 